In [1]:
# https://github.com/julianweisbord/pytorch-mlp-cifar10/blob/master/mlp.py

# Import PyTorch and other libraries
import torch
import torchvision

# Define the device to use (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the CIFAR-10 dataset and split it into train and test sets
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Define the MLP model with two hidden layers
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = torch.nn.Flatten() # Flatten the input images
        self.fc1 = torch.nn.Linear(3 * 32 * 32, 256) # First hidden layer with 256 neurons
        self.relu = torch.nn.ReLU() # ReLU activation function
        self.dropout = torch.nn.Dropout(0.2) # Dropout layer to prevent overfitting
        self.fc2 = torch.nn.Linear(256, 128) # Second hidden layer with 128 neurons
        self.fc3 = torch.nn.Linear(128, 10) # Output layer with 10 neurons
        self.softmax = torch.nn.Softmax(dim=1) # Softmax activation function

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x

model = MLP().to(device) # Create the model and move it to the device

# Define the loss function and the optimizer
criterion = torch.nn.CrossEntropyLoss() # Categorical crossentropy loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # SGD optimizer

# Train the model for 20 epochs
for epoch in range(20):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs and labels
        inputs, labels = data[0].to(device), data[1].to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        # Compute the loss
        loss = criterion(outputs, labels)
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # Print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

# Test the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

100%|██████████| 170498071/170498071 [01:19<00:00, 2132817.88it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
[1,  2000] loss: 2.293
[1,  4000] loss: 2.225
[1,  6000] loss: 2.186
[1,  8000] loss: 2.143
[1, 10000] loss: 2.124
[1, 12000] loss: 2.111
[2,  2000] loss: 2.093
[2,  4000] loss: 2.085
[2,  6000] loss: 2.077
[2,  8000] loss: 2.080
[2, 10000] loss: 2.072
[2, 12000] loss: 2.078
[3,  2000] loss: 2.057
[3,  4000] loss: 2.065
[3,  6000] loss: 2.054
[3,  8000] loss: 2.046
[3, 10000] loss: 2.055
[3, 12000] loss: 2.036
[4,  2000] loss: 2.031
[4,  4000] loss: 2.024
[4,  6000] loss: 2.028
[4,  8000] loss: 2.030
[4, 10000] loss: 2.022
[4, 12000] loss: 2.024
[5,  2000] loss: 2.011
[5,  4000] loss: 2.016
[5,  6000] loss: 2.009
[5,  8000] loss: 2.010
[5, 10000] loss: 2.006
[5, 12000] loss: 2.013
[6,  2000] loss: 1.992
[6,  4000] loss: 1.997
[6,  6000] loss: 1.993
[6,  8000] loss: 1.997
[6, 10000] loss: 2.003
[6, 12000] loss: 2.004
[7,  2000] loss: 1.982
[7,  4000] loss: 1.989
[7,  6000] loss: 1.991
[7,  8000] los

KeyboardInterrupt: 